In [2]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.utils import np_utils
from scipy import io
import numpy as np
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

import seaborn as sns
sns.set_style("white")

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
data = io.loadmat("/media/zuziel/Windows/Users/Zuza/Documents/Programowanie/keras/notMNIST_small.mat")

In [5]:
X = data['images']
y = data['labels']
resolution = 28
classes = 10

X = np.transpose(X, (2, 0, 1))
X = X.reshape(X.shape[0], 1, 28, 28)

y = y.astype('int32')
X = X.astype('float32') / 255.

# 3 -> [0., 0., 0., 2., 0., 0., 0., 0., 0., 0.]
Y = np_utils.to_categorical(y, 10)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

Sieć z trzema warstwami o rozmiarze 512

In [5]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu', dim_ordering="th"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
from keras_ascii_sequential import sequential_model_to_ascii_printout
sequential_model_to_ascii_printout(model)

      OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

          Input   #####   (1, 28, 28)
  Convolution2D    \|/  -------------------       832     0.0%
           relu   #####   (32, 24, 24)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (16, 12, 24)
        Dropout    | || -------------------         0     0.0%
                  #####   (16, 12, 24)
        Flatten   ||||| -------------------         0     0.0%
                  #####   (4608,)
          Dense   XXXXX -------------------   2359808    81.6%
           relu   #####   (512,)
          Dense   XXXXX -------------------    262656     9.1%
           relu   #####   (512,)
          Dense   XXXXX -------------------    262656     9.1%
           relu   #####   (512,)
          Dense   XXXXX -------------------      5130     0.2%
        softmax   #####   (10,)


In [8]:
# checkpoint
filepath="net5_weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(X_train, Y_train,
                    nb_epoch=10,
                    batch_size=32,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks_list)

Train on 14979 samples, validate on 3745 samples
Epoch 1/10
14979/14979 [==============================] - 36s - loss: 0.4287 - acc: 0.8701 - val_loss: 0.3220 - val_acc: 0.9081
Epoch 2/10
14979/14979 [==============================] - 34s - loss: 0.2291 - acc: 0.9287 - val_loss: 0.2744 - val_acc: 0.9191
Epoch 3/10
14979/14979 [==============================] - 35s - loss: 0.1795 - acc: 0.9436 - val_loss: 0.2435 - val_acc: 0.9292
Epoch 4/10
 5184/14979 [=========>....................] - ETA: 21s - loss: 0.1328 - acc: 0.9597

In [10]:
print(history.history)

{'val_loss': [0.32201690699293395, 0.27441962378047335, 0.24346471250216378, 0.23383358165482496, 0.25293296331039894, 0.27786393314401042, 0.285769010024849, 0.32455272685291275, 0.26587796015627635, 0.40442431700958043], 'val_acc': [0.9081441922563418, 0.91909212283044062, 0.92923898531375171, 0.93190921228304402, 0.93271028037383175, 0.93004005340453944, 0.93778371161548735, 0.92496662216288383, 0.93965287049399204, 0.92469959946595459], 'loss': [0.42870819641180075, 0.22913031584831425, 0.17951215022665171, 0.13661438619886576, 0.10962279195930454, 0.077241550210980725, 0.06725248320956842, 0.06179138772066417, 0.060817895764807069, 0.042013286169365061], 'acc': [0.87008478536617928, 0.92870018025235335, 0.94358768943187132, 0.95760731691432033, 0.96621937378997258, 0.97489819080045392, 0.97803591695039727, 0.98217504506308828, 0.98063956205354164, 0.98751585553505605]}


Okazuje się, że wyniki są lepsze niż wpoprzednich przypadkach - udało się uzyskać aż 93,965% na zbiorze testowym.

In [11]:
score = model.evaluate(X,Y)
print(score)

18720/18724 [============================>.] - ETA: 0s    64/18724 [..............................] - ETA: 15s

In [12]:
print(model.evaluate(X_test,Y_test))

3744/3745 [============================>.] - ETA: 0s  64/3745 [..............................] - ETA: 2s

In [ ]:
history = model.fit(X_train, Y_train,
                    nb_epoch=10,
                    batch_size=32,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks_list)

Train on 14979 samples, validate on 3745 samples
Epoch 1/10
14979/14979 [==============================] - 44s - loss: 0.0531 - acc: 0.9846 - val_loss: 0.3492 - val_acc: 0.9322
Epoch 2/10
14979/14979 [==============================] - 55s - loss: 0.0330 - acc: 0.9899 - val_loss: 0.3295 - val_acc: 0.9381
Epoch 3/10
14979/14979 [==============================] - 36s - loss: 0.0268 - acc: 0.9917 - val_loss: 0.3723 - val_acc: 0.9346
Epoch 4/10
14720/14979 [============================>.] - ETA: 0s - loss: 0.0355 - acc: 0.9893     

In [14]:
print(history.history)

{'val_loss': [0.34922379899965239, 0.32950269984521557, 0.3722918579717806, 0.3868384341444156, 0.34120223528477989, 0.40736927002399803, 0.33453466530865339, 0.43016854845116126, 0.36554147048708169, 0.41120456776119152], 'val_acc': [0.93217623497997326, 0.93805073431241659, 0.93457943925233644, 0.93110814419225629, 0.93297730307076099, 0.92736982643524701, 0.9393858477970628, 0.92710280373831777, 0.93457943925233644, 0.93377837116154871], 'loss': [0.053123844537411667, 0.033049712467136227, 0.026785444969616018, 0.035264955305572283, 0.0320439683203207, 0.025084678289697113, 0.023338327920266227, 0.024529767969300141, 0.032495855249285853, 0.025843189959293425], 'acc': [0.98464516990453299, 0.98985246011082184, 0.99172174377461775, 0.98931837906402298, 0.99085386207356962, 0.99278990586821547, 0.99312370652246473, 0.9930569463916149, 0.99138794312036849, 0.99299018626076507]}


Kolejne 10 iteracji niewiele zmieniło, ale w międzyczasie również padł podobnie wysoki wynik - 93,939%

In [6]:
best_model = Sequential()
best_model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu', dim_ordering="th"))
best_model.add(MaxPooling2D(pool_size=(2, 2)))
best_model.add(Dropout(0.2))
best_model.add(Flatten())
best_model.add(Dense(512, activation='relu'))
best_model.add(Dense(512, activation='relu'))
best_model.add(Dense(512, activation='relu'))
best_model.add(Dense(classes, activation='softmax'))
# Compile model
best_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
best_model.load_weights("net5_weights-improvement-94.hdf5")

In [7]:
print(best_model.evaluate(X,Y))
print(best_model.evaluate(X_train,Y_train))
print(best_model.evaluate(X_test,Y_test))

14979/14979 [==============================] - 9s     

# Poszukiwanie najczęściej mylonych liter

In [8]:
predictions = best_model.predict(X)

In [9]:
errors = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
for i in range(0,X.shape[0]):
    errors[np.argmax(Y[i])][np.argmax(predictions[i])] += 1
print(errors)

[[1849, 1, 1, 3, 1, 0, 4, 11, 1, 1], [2, 1843, 2, 4, 8, 2, 7, 4, 1, 0], [1, 0, 1835, 0, 15, 1, 20, 0, 1, 0], [5, 4, 2, 1845, 1, 0, 3, 3, 2, 8], [1, 2, 8, 0, 1849, 6, 5, 0, 2, 0], [1, 0, 4, 0, 8, 1850, 1, 2, 3, 3], [4, 2, 15, 1, 7, 6, 1836, 0, 0, 1], [9, 1, 1, 2, 1, 0, 4, 1848, 6, 0], [3, 1, 5, 8, 6, 0, 2, 3, 1820, 24], [2, 0, 2, 6, 0, 0, 2, 1, 21, 1838]]


Wynika z tego, że najczęściej mylone są:
 1. I z J (24 razy)
 2. J z I (21 razy)
 3. C z G (20 razy)
 4. C z E (15 razy)
  . G z C (15 razy)
 6. A z H (11 razy)